In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.losses import CategoricalFocalCrossentropy
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import layers
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix


# Directory
B3_RESULTS_DIR = "/content/drive/My Drive/efficientNet_B3"
os.makedirs(B3_RESULTS_DIR, exist_ok=True)

# Dataset paths
dataset_root = "/content/drive/My Drive/dataset"
train_dir = os.path.join(dataset_root, "train")
val_dir = os.path.join(dataset_root, "validation")
test_dir = os.path.join(dataset_root, "test")


image_size = (300, 300)
batch_size = 32

# Load datasets
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=image_size,
    batch_size=batch_size,
    label_mode="categorical",
    shuffle=True,
    seed=42,
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=image_size,
    batch_size=batch_size,
    label_mode="categorical",
    shuffle=False,
)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=image_size,
    batch_size=batch_size,
    label_mode="categorical",
    shuffle=False,
)

class_names = train_data.class_names


# Data Pipeline with Proper Preprocessing

def efficientnet_preprocess(image, label):
    image = tf.image.resize(image, image_size)
    image = tf.keras.applications.efficientnet.preprocess_input(image)
    return image, label

# Data augmentation
augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.05),

])

def augment_data(image, label):
    image = augmentation(image)
    return image, label

# Applying preprocessing and augmentation
train_data = train_data.map(efficientnet_preprocess)
train_data = train_data.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)

# Validation and test get preprocessing only
val_data = val_data.map(efficientnet_preprocess).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(efficientnet_preprocess).prefetch(tf.data.AUTOTUNE)


# Model Construction
def build_model():
    base_model = EfficientNetB3(
        include_top=False,
        weights='imagenet',
        input_shape=(*image_size, 3),
        pooling=None
    )

    # Freezing initial layers
    base_model.trainable = True
    for layer in base_model.layers[:150]:
        layer.trainable = False

    # Simplified head
    inputs = tf.keras.Input(shape=(*image_size, 3))
    x = base_model(inputs, training=True)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(len(class_names), activation='softmax')(x)

    return Model(inputs, outputs)

model = build_model()


# Training Configuration
# loss function
loss = CategoricalFocalCrossentropy(
    alpha=[0.25]*len(class_names),  # Balanced class weights
    gamma=2.0
)

# optimizer
optimizer = AdamW(
    learning_rate=3e-5,
    weight_decay=1e-4
)

# metrics
metrics = [
    'accuracy',
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(B3_RESULTS_DIR, 'best_model.keras'),
        monitor='val_accuracy',
        save_best_only=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5
    ),
    tf.keras.callbacks.CSVLogger(
        os.path.join(B3_RESULTS_DIR, 'training_log.csv'))
]

# Training
print("Starting training...")
initial_epochs = 30
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=initial_epochs,
    callbacks=callbacks
)

# Fine-Tuning Phase
print("\nStarting fine-tuning...")

# Unfreezinf all layers
for layer in model.layers:
    layer.trainable = True

# Recompile with lower learning rate
model.compile(
    optimizer=AdamW(learning_rate=1e-5, weight_decay=1e-5),
    loss=loss,
    metrics=metrics
)

# Fine-tune for fewer epochs
fine_tune_epochs = 15
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(
    train_data,
    validation_data=val_data,
    initial_epoch=history.epoch[-1] + 1,
    epochs=total_epochs,
    callbacks=callbacks
)


# Evaluation
def evaluate_model(model, dataset):
    y_true, y_pred = [], []
    for images, labels in dataset:
        y_true.extend(tf.argmax(labels, axis=1).numpy())
        y_pred.extend(tf.argmax(model.predict(images, verbose=0), axis=1))

    # Classification report
    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    pd.DataFrame(report).transpose().to_csv(os.path.join(B3_RESULTS_DIR, 'classification_report.csv'))

    # Confusion matrix
    plt.figure(figsize=(15, 12))
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xticks(rotation=45, ha='right')
    plt.savefig(os.path.join(B3_RESULTS_DIR, 'confusion_matrix.png'))
    plt.close()

    return report

# Evaluate
test_report = evaluate_model(model, test_data)


# Visuals
def plot_history(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Curves')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Curves')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(B3_RESULTS_DIR, 'training_history.png'))
    plt.close()

plot_history(history_fine)


# Final Model
model.save(os.path.join(B3_RESULTS_DIR, 'final_model.keras'))

# TFLite conversion for android studio app
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open(os.path.join(B3_RESULTS_DIR, 'model.tflite'), 'wb') as f:
    f.write(tflite_model)


# Final Output
print("\nTraining completed successfully!")
print(f"Test Accuracy: {test_report['accuracy']:.4f}")
print(f"Macro F1-Score: {test_report['macro avg']['f1-score']:.4f}")
print(f"Results saved to: {B3_RESULTS_DIR}")

Mounted at /content/drive
Found 1360 files belonging to 17 classes.
Found 170 files belonging to 17 classes.
Found 170 files belonging to 17 classes.
43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Starting training...
Epoch 1/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 473s 2s/step - accuracy: 0.0541 - auc: 0.5189 - loss: 0.6429 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.1765 - val_auc: 0.6863 - val_loss: 0.5751 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 3.0000e-05
Epoch 2/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 14s 325ms/step - accuracy: 0.3291 - auc: 0.8046 - loss: 0.5067 - precision: 0.0682 - recall: 5.0536e-05 - val_accuracy: 0.4765 - val_auc: 0.8848 - val_loss: 0.4454 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 3.0000e-05
Epoch 3/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 16s 365ms/step - accuracy: 0.6214 - auc: 0.9373 - loss: 0.3670 - precision: 0.9318 - recall: 0.0239 - val_accuracy: 0.6706 - val_auc: 0.9588 - val_loss: 0.3056 - val_precis